In [ ]:
import infostop
import pandas as pd
import numpy as np

import osmnx as ox
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline
ox.config(use_cache=True, log_console=True)
print(ox.__version__)

import networkx as nx
import geopandas as gpd
import multiprocessing as mp

from descartes import PolygonPatch
from shapely.geometry import Polygon, MultiPolygon

import folium
from folium.plugins import Fullscreen, HeatMapWithTime, TimestampedGeoJson
from folium.plugins import TimestampedGeoJson, HeatMap, HeatMapWithTime

import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import plotly_express as px
import tqdm
from tqdm._tqdm_notebook import tqdm_notebook

data = pd.read_csv('data_one_month.csv')
data.head()

#### create model
model = infostop.Infostop()

#### data need to be converted in order to be used in infostop 
data = data.sort_values(by = ['timestamp', 'user'], ascending=True)

#### convert columns to numpy array 
gps_data = data[['lat', 'lon', 'timestamp']].to_numpy()

#### FIND STOPS FOR EACH USER. GROUPBY USER TO FIND STOPS.
labels = model.fit_predict(gps_data)

folmap = infostop.plot_map(model)
folmap.m

# convert to list 
labels_list = labels.tolist()
data['labels'] = labels_list 
data.head()

data = pd.DataFrame(infostop.postprocess.compute_intervals(data[['lat','lon','timestamp']].values,labels),
            columns = ['label','start','end','lat','lon'])
data.head()

- FIND STOPS FOR EACH USER. GROUPBY USER TO FIND STOPS.

#### non-stationary points are assigned to -1, stationary are assigned to a positive integer
stops = data[data['label'] != -1]
stops = stops.reset_index(drop=True)
stops.head()

stops.to_csv('stops.csv', index=False)

del data, stops

In [ ]:
from IPython.display import display, HTML, Markdown

def print_df(df):
    return display(HTML(df.to_html()))

stops = pd.read_csv('stops.csv')

print_df(stops.head())

In [ ]:
stops.rename(columns={'loc':'label'}, inplace=True)
stops['start'] = pd.to_datetime(stops['start'], unit='ms')
stops['end'] = pd.to_datetime(stops['end'], unit='ms')
stops['label'] = stops['label'].apply(int)
stops.head()

In [ ]:
stops.shape

In [ ]:
stops['label'] = stops['label'].apply(str)
stops['user'] = stops['user'].apply(str)
stops.head()

In [ ]:
stops.dtypes

In [ ]:
len(stops['label'].unique())

In [ ]:
stops.tail()

In [ ]:
print(stops.loc[(stops['start'] >= pd.to_datetime('2014-01-01')) & (stops['start'] < pd.to_datetime('2014-01-02')) & (stops['user'] == '0')].shape)
print_df(stops.loc[(stops['start'] >= pd.to_datetime('2014-01-01')) & (stops['start'] < pd.to_datetime('2014-01-02')) & (stops['user'] == '0')].head())

In [ ]:
oneday = stops.loc[(stops['start'] >= pd.to_datetime('2014-01-01')) & (stops['start'] < pd.to_datetime('2014-01-02')) & ((stops['user'] == '0') | (stops['user'] == '1') | (stops['user'] == '2'))].copy()
oneday.head()

In [ ]:
display(Markdown("<font color=green> <font size=4>'oneday' dataframe has 3 people's information which has the user labels of 0, 1 and 2.\
                 \nShape of this dataframe is {}.".format(oneday.shape)))

In [ ]:
loc_data = [[row['lat'],row['lon']] for index, row in oneday.iterrows()]

In [ ]:
map_hooray = folium.Map(location=[55.636413, 11.298542], zoom_start = 3, tiles='Stamen Toner')
HeatMap(loc_data, radius = 20, max_zoom = 30).add_to(map_hooray)
map_hooray

In [ ]:
oneday_median = pd.merge(oneday.groupby(by=['label'])['lat'].median().reset_index().copy(), \
                         oneday.groupby(by=['label'])['lon'].median().reset_index().copy(), how='outer', on='label')
oneday_median.rename(columns={'lat':'lat_median', 'lon':'lot_median'}, inplace=True)
oneday_median

In [ ]:
def percent_missing(df):
    percent_missing = df.isnull().sum() * 100 / len(df)
    missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})
    missing_value_df.sort_values('percent_missing', inplace=True)
    missing_value_df.reset_index(drop=True, inplace=True)
    return missing_value_df

missing_df_median = percent_missing(oneday_median)
missing_df_median

In [ ]:
print('{} rows dropped after finding median latitudes and longitudes for each label.'.format(oneday.shape[0]-oneday_median.shape[0]))

In [ ]:
oneday_median.dtypes

In [ ]:
from geopy.distance import geodesic

coords_1 = (52.2296756, 21.0122287)
coords_2 = (52.406374, 16.9251681)

print('Distance Miles:      ', geodesic(coords_1, coords_2).miles)
print('Distance Kilometers: ', geodesic(coords_1, coords_2).km)
print('Distance Meters:     ', geodesic(coords_1, coords_2).m)

In [ ]:
loc_data = [[row['lat'],row['lon']] for index, row in oneday_median.iterrows()]
map_hooray = folium.Map(location=[55.636413, 11.298542], zoom_start = 3, tiles='Stamen Toner')
HeatMap(loc_data, radius = 20, max_zoom = 30).add_to(map_hooray)
map_hooray

In [ ]:
oneday_real = pd.merge(oneday, oneday_median, on='label', how='left')
oneday_real.sort_values(by=['start', 'end', 'label'], inplace=True)
oneday_real.reset_index(drop=True, inplace=True)
print_df(oneday_real.head())

<font color = green>
<font size = 4>

- Now, median coordinates are found for each label.
- Only media coordinates will be kept and others will be dropped.

In [ ]:
oneday_real.drop(columns=['lat', 'lon'], inplace=True)
oneday_real.rename(columns={'lat_median':'lat', 'lot_median':'lon'}, inplace=True)
oneday_real["geo"] = oneday_real["lat"].map(str) + ", " + oneday_real["lon"].map(str)
oneday_real.head()

In [ ]:
%who DataFrame

In [ ]:
del oneday_median
oneday_median = oneday_real.copy()
del missing_df_median, oneday, oneday_real

In [ ]:
missing_df_median = percent_missing(oneday_median)
missing_df_median

In [ ]:
del missing_df_median
oneday_median

In [ ]:
# 'place' is the first row of 'oneday_median' dataset
place = tuple((float(oneday_median['geo'][0].split(', ')[0]), float(oneday_median['geo'][0].split(', ')[1])))

tags = {'amenity' : True,
        'landuse' : ['retail', 'commercial'],
        'highway' : 'bus_stop'}
gdf = ox.geometries_from_point(center_point = place, tags = tags)
gdf.shape

In [ ]:
pd.DataFrame(gdf)

- We should find a surrounding box of any points.

<font color = green>
<font size = 4>

- I don't need following code to find out outliers in data since I know my data based on user label.
- Therefore, I turned them in Markdown

len(oneday_median['label'].unique())

- I will drop the outliers from the data in order to create a bounding box from lat and lon

plt.hist(bins=20, x=oneday_median['lat'])
plt.show()

plt.hist(bins=20, x=oneday_median['lon'])
plt.show()

oneday_median.drop(index=oneday_median.loc[oneday_median['lon']>25].index, inplace=True)

plt.hist(bins=20, x=oneday_median['lat'])
plt.show()

plt.hist(bins=20, x=oneday_median['lon'])
plt.show()

oneday_median.loc[oneday_median['lon']<5]

oneday_median.drop(index=list(oneday_median.loc[oneday_median['lon']<5].index), inplace=True)

plt.hist(bins=20, x=oneday_median['lon'])
plt.show()

plt.hist(bins=20, x=oneday_median['lat'])
plt.show()

oneday_median.loc[(oneday_median['lat']>57) | (oneday_median['lat']<54)]

oneday_median.drop(index=list(oneday_median.loc[(oneday_median['lat']>57) | (oneday_median['lat']<54)].index), inplace=True)

plt.hist(bins=20, x=oneday_median['lat'])
plt.show()

plt.hist(bins='auto', x=oneday_median['lon'])
plt.show()

Since latitude histogram seems a little more normal distributed, I will stick to that.

plt.hist(bins='auto', x=oneday_median['lat'])
plt.show()

oneday_median.reset_index(drop=True, inplace=True)

In [ ]:
print(
oneday_median['lat'].min(),
oneday_median['lat'].max(),
oneday_median['lon'].min(),
oneday_median['lon'].max())

- After deleting outliers, a more meaningful map is visualized below.

In [ ]:
loc_data = [[row['lat'],row['lon']] for index, row in oneday_median.iterrows()]
map_hooray = folium.Map(location=[55.636413, 11.298542], zoom_start = 3, tiles='Stamen Toner')
HeatMap(loc_data, radius = 20, max_zoom = 30).add_to(map_hooray)
map_hooray

In [ ]:
south = oneday_median['lat'].min()
north = oneday_median['lat'].max()
west = oneday_median['lon'].min()
east = oneday_median['lon'].max()
tags = {'amenity' : True,
        'landuse' : ['retail', 'commercial'],
        'highway' : 'bus_stop'}
gdf = ox.geometries_from_bbox(north = north, south = south, east = east, west = west, tags = tags)
gdf.shape

In [ ]:
gdf[gdf['amenity']=='bank'].dropna(axis=1, how='any')

<font color = green>
<font size = 4>

- Skip the following codes till find a warning again in a Markdown cell.

In [ ]:
gdf[gdf['highway']=='bus_stop'].dropna(axis=1, how='any').head()

In [ ]:
ax = gdf.plot()
_ = ax.axis('off')

In [ ]:
ax = gdf.plot()

In [ ]:
from functools import partial
import pyproj
from shapely.ops import transform
from shapely.geometry import Point

In [ ]:
proj_wgs84 = pyproj.Proj('+proj=longlat +datum=WGS84')


def geodesic_point_buffer(lat, lon, km):
    # Azimuthal equidistant projection
    aeqd_proj = '+proj=aeqd +lat_0={lat} +lon_0={lon} +x_0=0 +y_0=0'
    project = partial(
        pyproj.transform,
        pyproj.Proj(aeqd_proj.format(lat=lat, lon=lon)),
        proj_wgs84)
    buf = Point(0, 0).buffer(km * 1000)  # distance in metres
    return transform(project, buf).exterior.coords[:]

# Example
# 20 meters
circle = geodesic_point_buffer(55.636413, 11.298542, 10)
circle = [list(i) for i in circle]
for i in range(len(circle)):
    circle[i].reverse()
circle

- Longitude indexed sorting above.

In [ ]:
maps = folium.Map(location=circle[0], tiles='Stamen Toner',zoom_start=9)
for i in range(len(circle)):
    folium.CircleMarker(location=circle[i], radius=4, color='darkred', fill=True, fill_color='lightblue',\
                       popup="{}".format("Text"), icon=folium.Icon(color='green')).add_to(maps)
maps

In [ ]:
oneday_median

- It is not gonna be a circle but it will include 20 meter diameter circle since the edges of the bounding box will be 20 meters.

In [ ]:
def surrounding_box(lat, lon, km = 0.03):
    # 30 meters as a default
    
    max_lat = 0
    min_lat = 999999
    max_lon = 0
    min_lon = 999999

    circle = geodesic_point_buffer(lat, lon, km)
    circle = [list(i) for i in circle]

    for i in range(len(circle)):
        circle[i].reverse()
        if circle[i][0] > max_lat:
            max_lat = circle[i][0]
        if circle[i][0] < min_lat:
            min_lat = circle[i][0]
        if circle[i][1] > max_lon:
            max_lon = circle[i][1]
        if circle[i][1] < min_lon:
            min_lon = circle[i][1]
    
    return min_lat, max_lat, min_lon, max_lon

In [ ]:
oneday_median[['min_lat', 'max_lat', 'min_lon', 'max_lon']] = oneday_median.apply(\
                                                        lambda row: surrounding_box(row['lat'], row['lon']), axis=1, result_type="expand")

In [ ]:
oneday_median

In [ ]:
gdf[gdf['amenity']=='bank'].dropna(axis=1, how='any')

In [ ]:
gdf_df = pd.DataFrame(gdf)

In [ ]:
import warnings
warnings.filterwarnings('ignore')
gdf_df.shape

In [ ]:
print_df(gdf_df.head())

In [ ]:
list(gdf_df.columns)

In [ ]:
gdf_df['geometry'].dtypes

In [ ]:
gdf_df['geometry'][0]

In [ ]:
gdf_df.loc[gdf_df['geometry'] == 'POINT (8.47627 55.46630)'].dropna(axis=1, how='any')

In [ ]:
def turn_geometry_values(row):
    return np.asarray(row)[1], np.asarray(row)[0]

# , axis=1, result_type="expand"

In [ ]:
print(gdf_df['geometry'][77861].centroid.y)

In [ ]:
gdf_df['center_point'] = gdf_df['geometry'].map(lambda row: row.centroid)
gdf_df["lat"] = gdf_df.center_point.map(lambda row: row.y)
gdf_df["lon"] = gdf_df.center_point.map(lambda row: row.x)

In [ ]:
print_df(gdf_df[['geometry', 'center_point', 'lat', 'lon']].head())

In [ ]:
print(list(np.asarray(gdf_df.geometry.values[0])))

gdf[gdf['amenity']=='bank'].dropna(axis=1, how='any')
gdf[gdf['highway']=='bus_stop'].dropna(axis=1, how='any').head()

In [ ]:
oneday_median.head()

In [ ]:
print_df(gdf_df.head())

- First I designed dataframes to compare in pandas but I realized that it is easier in Geopandas.

<font color = green>
<font size = 4>

- Start from here after 'oneday_median' is ready.

In [ ]:
oneday_gdf = gpd.GeoDataFrame(
    oneday_median, geometry=gpd.points_from_xy(oneday_median.lon, oneday_median.lat))

In [ ]:
oneday_gdf.head()

In [ ]:
gdf['center_point'] = gdf['geometry'].map(lambda row: row.centroid)
gdf["gdf_lat"] = gdf.center_point.map(lambda row: row.y)
gdf["gdf_lon"] = gdf.center_point.map(lambda row: row.x)

In [ ]:
print_df(gdf[['geometry', 'center_point', 'gdf_lat', 'gdf_lon']].head())

In [ ]:
#https://gis.stackexchange.com/questions/222315/geopandas-find-nearest-point-in-other-dataframe
#https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.BallTree.html#
#https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html

# BALL TREES, distance_upper_bound, haver sine distance, haversine

from sklearn.neighbors import BallTree
from shapely.geometry import Point
import functools
import operator

# find that distances, if they are out of 20meters
# For a specified leaf_size, a leaf node is guaranteed to satisfy;
# leaf_size <= n_points <= 2 * leaf_size, except in the case that n_samples < leaf_size.
def ball_nearest(df, gdf):
    array_DF = np.array(list(df.geometry.apply(lambda x: (x.x, x.y))))
    array_GDF = np.array(list(gdf.center_point.apply(lambda x: (x.x, x.y))))
    # Tree will be created according to our gdf_df since the closest points will be found out according to these 'triangulations'
    leafSize = round(len(array_GDF)) # to guarantee number of leaves
    btree = BallTree(array_GDF, metric='haversine', leaf_size=leafSize)
    # Query will be conducted for the 'User' data.
    dist, idx = btree.query(array_DF, k=1)
    idx = functools.reduce(operator.iconcat, idx, [])
    dist = functools.reduce(operator.iconcat, dist, [])
    gdf = pd.concat([df.reset_index(drop=True), gdf.loc[idx, gdf.columns != 'geometry'].reset_index(drop=True), pd.Series(dist, name='dist')], axis=1)
    
    return gdf, dist, idx

nearest_gdf, distances, indices = ball_nearest(oneday_gdf, gdf)

nearest_gdf

In [ ]:
print_df(nearest_gdf.head().dropna(axis=1, how='any'))

In [ ]:
list(nearest_gdf['amenity'])

<font color = green>
<font size = 4>

- __*'dist'*__ is the difference between two latitude and longitude sets. (2 coordinates). 
- How should I convert __*'dist'*__ to __*'meters'*__?
- __*'haversine'*__ outputs in __*'radians'*__

In [ ]:
print_df(nearest_gdf.head().dropna(axis=1, how='any'))

In [ ]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

haversine(nearest_gdf.loc[0, 'lon'], nearest_gdf.loc[0, 'lat'], nearest_gdf.loc[0, 'gdf_lon'], nearest_gdf.loc[0, 'gdf_lat'])

In [ ]:
from sklearn.metrics.pairwise import haversine_distances
from math import radians

row0 = nearest_gdf.loc[0, 'lat'], nearest_gdf.loc[0, 'lon']
row0_0 = nearest_gdf.loc[0, 'gdf_lat'], nearest_gdf.loc[0, 'gdf_lon']
row0_in_radians = [radians(_) for _ in row0]
row0_0_in_radians = [radians(_) for _ in row0_0]
result = haversine_distances([row0_in_radians, row0_0_in_radians])
print('Result in radians:\n',result)
print('Result in km:\n',result * 6371000/1000)

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.haversine_distances.html#:~:text=The%20Haversine%20(or%20great%20circle,the%20longitude%2C%20given%20in%20radians.
nearest_gdf.loc[0, 'dist'] * 6371000/1000  # multiply by Earth radius to get kilometers

<font color = green>
<font size = 4>

- __*'dist'*__ isn't same with the ones I have found as in above;